<a href="https://colab.research.google.com/github/Artem33244/.github/blob/main/Dont_Have_RTX_4090%3F_Try_This!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Пайп лайны взяты https://huggingface.co/docs/diffusers
#Обязательно в среде выполнение поставьте себе GPU перед началом
!pip install diffusers["torch"] transformers
!pip install accelerate
!pip install git+https://github.com/huggingface/diffusers

### **ТЕКСТ В ИЗОБРАЖЕНИЕ**

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
# Вместо "redstonehero/cetusmix_v4" вставляется название модели. В основном найти модели можно на сайте https://civitai.com/ от туда уже ищите модель на сайте https://huggingface.co/,
# Когда зашли на нужную нам модель на хаге в ссылке страницы вы увидите https://huggingface.co/ByteDance/SDXL-Lightning
# из ссылки нам надо получить ByteDance/SDXL-Lightning и вставить сюда вместо redstonehero/cetusmix_v4
pipe = StableDiffusionPipeline.from_pretrained("redstonehero/cetusmix_v4", torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.safety_checker = None
#То что вы хотите получить, то что в начале строки генерируется первым, можно использовать скобки (word) ((word)) (((word))) для изменения приоритета внутри запроса
prompt = "medival fortress, landscape, scenery, 8K, masterpiece, ultradetailed"
# Высота и ширина рекомендуется не делать слишком большое.
h=800
w=640
# Количество шагов по созданию изображения. Чем их больше, тем дольше нейросеть будет над ней работать но тем более детализированее будет изображение, лучше не выходить за пределы 40 шагов, может дать обратный эффект.
steps=25
# Точность, чем выше тем больше точность генерации, рекомендуется оставить на значении 7,5
guidance=7.5
# То что вы не хотите получить на фотографии
neg = "easynegative, human, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worstquality, low quality, normal quality, jpegartifacts, signature, watermark, username, blurry, bad feet, cropped, poorly drawn hands, poorly drawn face, mutation, deformed, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, extra fingers, fewer digits, extra limbs, extra arms,extra legs, malformed limbs, fused fingers, too many fingers, long neck, cross-eyed,mutated hands, polar lowres, bad body, bad proportions, gross proportions, text, error, missing fingers, missing arms, missing legs, extra digit, extra arms, extra leg, extra foot,"

image = pipe(prompt, height=h, width=w, num_inference_steps=steps, guidance_scale=guidance, negative_prompt=neg).images[0]
image

### **ИЗОБРАЖЕНИЕ В ИЗОБРАЖЕНИЕ**

In [ ]:
import torch
import numpy as np

from transformers import pipeline
from diffusers.utils import load_image, make_image_grid

image = load_image(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/controlnet-img2img.jpg" # Вставьте сюда ссылку на ваше изображение
)

def get_depth_map(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    detected_map = torch.from_numpy(image).float() / 255.0
    depth_map = detected_map.permute(2, 0, 1)
    return depth_map

depth_estimator = pipeline("depth-estimation")
depth_map = get_depth_map(image, depth_estimator).unsqueeze(0).half().to("cuda")

In [ ]:
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, UniPCMultistepScheduler
import torch

controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11f1p_sd15_depth", torch_dtype=torch.float16, use_safetensors=True)
pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16, use_safetensors=True # На этой строчке вы можете подгрузить модель как в примере выше
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

In [ ]:
output = pipe(
    "lego batman and robin", image=image, control_image=depth_map, ## ПРОМПТ К ИЗОБРАЖЕНИЮ
).images[0]
make_image_grid([image, output], rows=1, cols=2)